In [ ]:
!git clone https://github.com/AbdullahO/SAMoSSA.git
import numpy as np
dataset = np.load('/content/SAMoSSA/datasets/electricity/electricity.npy', encoding='bytes')

In [ ]:
training_set = dataset[25800:25848]      # Arrays 1 to 25824
validation_set = dataset[25848:25872] # Arrays 25825 to 25872

In [ ]:
sampled_user_ids=[ 58,  53,  84, 274, 164, 365, 340, 225, 281,  48,  42, 298, 334,
        63,   3, 229, 262, 104,  64,  27, 133,  61, 245,   2,  67, 337,
       127, 248, 218, 217, 317, 280, 243,  76, 219, 250, 305,  75, 350,
        49,  95, 224, 162, 367,  73, 161, 238, 324,  29, 154]

In [ ]:
!pip install tensorly

In [ ]:
from tensorly.decomposition import robust_pca

#Training phase
-Create blue matrix \\
-SVD decomposition -> find orange matrix (non-stationary part) \\
-Subtract blue from orange -> get green matrix -> stationary part/residual \\
-Calculate beta_hat (will be used in testing phase for out of sample forecasting of "SAMoSSA" part

In [ ]:
import pandas as pd
import numpy as np

number_of_hours, num_users = training_set.shape

# Generate date range
date_range = pd.date_range(start='01/01/2011 00:00', periods=number_of_hours, freq='H')

# Reshape and create pairs of values and user IDs
data = []
for user_id in range(1, num_users + 1):
    for hour, value in enumerate(training_set[:, user_id - 1]):
        data.append([date_range[hour], value, user_id])

# Create DataFrame
df_train = pd.DataFrame(data, columns=['Date', 'Load', 'UserID'])


In [ ]:
import pandas as pd
import numpy as np

number_of_hours, num_users = validation_set.shape

# Generate date range
date_range = pd.date_range(start='17/01/2011 00:00', periods=number_of_hours, freq='H')

# Reshape and create pairs of values and user IDs
data = []
for user_id in range(1, num_users + 1):
    for hour, value in enumerate(validation_set[:, user_id - 1]):
        data.append([date_range[hour], value, user_id])

# Create DataFrame
df_valid = pd.DataFrame(data, columns=['Date', 'Load', 'UserID'])


In [ ]:
# Filter the original DataFrame to include only the sampled user IDs
df_train = df_train[df_train['UserID'].isin(sampled_user_ids)]
df_valid = df_valid[df_valid['UserID'].isin(sampled_user_ids)]

In [ ]:
# Resetting the index
df_train = df_train.reset_index(drop=True)
df_train=df_train[['Load', 'UserID']]

In [ ]:
arrival_times_per_user = df_train.groupby('UserID')['Load'].apply(list)
arrival_times_per_user

In [ ]:
def list_to_matrix_columnwise_corrected(lst, rows=48, columns=1):
    # Initialize a matrix of zeros
    matrix = np.zeros((rows, columns))

    # Fill the matrix column-wise
    for i, val in enumerate(lst):
        row = i % rows
        col = i // rows
        if col < columns:
            matrix[row, col] = val

    return matrix

page_matrices = {user_id: list_to_matrix_columnwise_corrected(times) for user_id, times in arrival_times_per_user.items()}


In [ ]:
matrices = list(page_matrices.values())

# Stacking the matrices horizontally
stacked_page_matrix = np.hstack(matrices)
stacked_page_matrix.shape ##blue matrix created

In [ ]:
stacked_page_matrix

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
data_standardized = scaler.fit_transform(stacked_page_matrix)  # Transpose to standardize across users, not time points


In [ ]:
low_rank_part, sparse_part = robust_pca(data_standardized, reg_E=0.04, learning_rate=1.2, n_iter_max=100)

In [ ]:
L_inverse_scaled = scaler.inverse_transform(low_rank_part)

In [ ]:
L_inverse_scaled

##SVD decomposition
-Use a hard margin of k=5 (same as the one used in the paper)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Assuming 'stacked_page_matrix' is your data matrix

# Step 1: PCA automatically centers the data, so you don't need to manually subtract the mean

# Step 2: Perform PCA with the desired number of components
num_principal_components = 10  # This is akin to your num_singular_values
pca = PCA(n_components=num_principal_components)

# Fit PCA to the data and transform the data onto the principal components
pca.fit(L_inverse_scaled)
transformed_data = pca.transform(L_inverse_scaled)

# Step 3: Reconstruct the data from the principal components
non_stationary_component = pca.inverse_transform(transformed_data)

# The 'reconstructed_data' matrix now acts as your non-stationary component


In [ ]:
pca.explained_variance_ratio_

In [ ]:
from scipy.linalg import svd

# The singular values are in the vector 's'
# U and VT are the left and right singular vectors, respectively
U, s, VT = svd(low_rank_part)
num_singular_values = 10
s_reduced = np.zeros(low_rank_part.shape)
np.fill_diagonal(s_reduced, s[:num_singular_values])

non_stationary_component = U @ s_reduced @ VT ##orange matrix

In [ ]:
num_singular_values = 10
s_reduced = np.zeros(stacked_page_matrix.shape)
np.fill_diagonal(s_reduced, s[:num_singular_values])

non_stationary_component = U @ s_reduced @ VT ##orange matrix

In [ ]:
from scipy.linalg import svd
# Mean centering the data
stacked_page_matrix_centered = stacked_page_matrix - np.mean(stacked_page_matrix, axis=0)

# Applying SVD to the mean-centered data
U, s, VT = svd(stacked_page_matrix_centered)

num_singular_values = 10
s_reduced = np.zeros(stacked_page_matrix.shape)
np.fill_diagonal(s_reduced, s[:num_singular_values])

# Reconstruction from the reduced number of singular values
non_stationary_component_centered = U @ s_reduced @ VT

# Adding back the mean to the centered reconstruction
non_stationary_component = non_stationary_component_centered + np.mean(stacked_page_matrix, axis=0)

In [ ]:
F_hat_mat = non_stationary_component[:47, :] #L-1 rows of orange matrix
y_vector = stacked_page_matrix[-1, :] #last row of blue matrix

import numpy as np

Y = y_vector.reshape(-1, 1)  # Reshape Y to be a column vector if it's a 1D array

# Solve for beta_hat using the least squares method
beta_hat, residuals, rank, s = np.linalg.lstsq(F_hat_mat.T, Y, rcond=None)

# # beta_hat now contains the estimated beta parameters, should be 18x1 dimensions
beta_hat

In [ ]:
array=beta_hat
desired_length = 48
last_element = array[-1, :]
while len(array) < desired_length:
    array = np.vstack([array, last_element])

In [ ]:
residual=stacked_page_matrix-array.T@non_stationary_component ##green matrix


In [ ]:
residual

##Train LSTM with residual (one LSTM model per user)

In [ ]:
userIDs = list(arrival_times_per_user.keys())
residuals_dict = {userID: residual[:, i] for i, userID in enumerate(userIDs)}


In [ ]:
residuals_dict

In [ ]:
import numpy as np

# Function to scale data to [-1, 1] range
def scale_to_neg_one_to_one(X, X_min, X_max):
    return 2 * ((X - X_min) / (X_max - X_min)) - 1

scaled_residuals_dict = {}
mins_dict = {}
maxs_dict = {}

# Iterate over each user's data in the dictionary
for key, values in residuals_dict.items():
    # Calculate the min and max from the current user's data
    X_min = np.min(values)
    X_max = np.max(values)

    # Scale the current user's data
    scaled_values = scale_to_neg_one_to_one(values, X_min, X_max)

    # Store the scaled values in the scaled dictionary
    scaled_residuals_dict[key] = scaled_values

    # Store the min and max values separately for later use (e.g., scaling test data)
    mins_dict[key] = X_min
    maxs_dict[key] = X_max

# scaled_data_dict now contains the scaled datasets for each user
# min_max_values_dict contains the min and max values for each user's data


In [ ]:
normalized_residuals_dict = {}
means_dict = {}
stds_dict = {}

for key, values in residuals_dict.items():
    # Calculate mean and standard deviation
    mean = values.mean()
    std = values.std()

    # Normalize values
    normalized_values = (values - mean) / std

    # Store normalized values and statistics
    normalized_residuals_dict[key] = normalized_values
    means_dict[key] = mean
    stds_dict[key] = std

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
from keras import backend as K

def smape_loss(y_true, y_pred):
    denominator = K.maximum(K.abs(y_true) + K.abs(y_pred), K.epsilon())
    diff = K.abs(y_pred - y_true) / denominator
    return 100 * K.mean(diff, axis=-1)

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=32, return_sequences=True, input_shape=input_shape))
    #model.add(Dropout(0.2))  # Dropout layer after the first LSTM layer
    model.add(LSTM(units=16, return_sequences=True))  # Second LSTM layer with 32 units
    #model.add(Dropout(0.2))  # Dropout layer after the second LSTM layer
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss=smape_loss)
    return model

def prepare_data(residuals, n_steps):
    X, y = [], []
    for i in range(len(residuals)):
        end_ix = i + n_steps
        if end_ix > len(residuals)-1:
            break
        seq_x, seq_y = residuals[i:end_ix], residuals[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

user_ids = normalized_residuals_dict.keys()
n_steps = 1  # Number of time steps for LSTM. Adjust as needed.

lstm_models = {}

for user_id in user_ids:
    residuals = normalized_residuals_dict[user_id]

    # Check if the user has sufficient data
    if len(residuals) > n_steps:
        X, y = prepare_data(residuals, n_steps)
        if X.size > 0 and y.size > 0:
            X = X.reshape((X.shape[0], X.shape[1], 1))
            lstm_model = create_lstm_model((X.shape[1], 1))
            lstm_model.fit(X, y, epochs=100, batch_size=1)
            lstm_models[user_id] = lstm_model
        else:
            print(f"Insufficient data for user {user_id}")
    else:
        print(f"Not enough data points for user {user_id} for n_steps = {n_steps}")


#Inference phase

##Estimate b_hat (parameters of non-stationary component)

In [ ]:
F_hat_mat = non_stationary_component[:48, :] #L-1 rows of orange matrix
y_vector = stacked_page_matrix[-1, :] #last row of blue matrix

import numpy as np

Y = y_vector.reshape(-1, 1)  # Reshape Y to be a column vector if it's a 1D array

# Solve for beta_hat using the least squares method
beta_hat, residuals, rank, s = np.linalg.lstsq(F_hat_mat.T, Y, rcond=None)

# # beta_hat now contains the estimated beta parameters, should be 18x1 dimensions


##Prediction of non-stationary part (using SAMoSSA method)
-Use method outlined in part V of figure (to predict value at time interval t use previous L points)

In [ ]:
import pandas as pd

# Initialize an empty dictionary to store time values for each user
user_times = {}

# Process the train dataset
for index, row in df_train.iterrows():
    user_id = row['UserID']
    load = row['Load']
    if user_id not in user_times:
        user_times[user_id] = []
    user_times[user_id].append(load)

# Process the test dataset
for index, row in df_valid.iterrows():
    user_id = row['UserID']
    load = row['Load']
    if user_id not in user_times:
        user_times[user_id] = []
    user_times[user_id].append(load)
#Create a dictionary where for each user store the last 18 values of the train dataset and the values
#of the test dataset for each user

In [ ]:
len(user_times[367.0])

In [ ]:
import numpy as np

# Initialize a dictionary to store predictions for each user
user_predictions_non_stationary = {}

for user_id, times in user_times.items():
    # We can only make a prediction if there are at least 18 values
    if len(times) >= 48:
        predictions = []
        # Slide the window and predict
        for i in range(len(times) - 48):
            window = np.array(times[i:i+48])
            prediction = np.dot(window, beta_hat).item()
            predictions.append(prediction)
        user_predictions_non_stationary[user_id] = predictions

# user_predictions now contains the predicted values for each user


In [ ]:
# Group by 'userID' and aggregate the 'time' values into lists
user_actual_time = df_valid.groupby('UserID')['Load'].apply(list).to_dict()

In [ ]:
# Assuming user_actual_time and user_predictions_non_stationary are dictionaries with lists as values
residual_test = {key: [a - b for a, b in zip(user_actual_time[key], user_predictions_non_stationary[key])]
                 for key in user_actual_time
                 if key in user_predictions_non_stationary}


In [ ]:
mins_dict

In [ ]:
import numpy as np

scaled_residuals_test={}

# Function to scale data to [-1, 1] range
def scale_to_neg_one_to_one(X, X_min, X_max):
    return 2 * ((X - X_min) / (X_max - X_min)) - 1


# Iterate over each user's data in the dictionary
for key, values in residual_test.items():

    X_min = mins_dict[key]
    X_max = maxs_dict[key]

    # Scale the current user's data
    scaled_values = scale_to_neg_one_to_one(values, X_min, X_max)

    # Store the scaled values in the scaled dictionary
    scaled_residuals_test[key] = scaled_values

# scaled_data_dict now contains the scaled datasets for each user
# min_max_values_dict contains the min and max values for each user's data

In [ ]:
# Adjusted dictionary to store the results
adjusted_residual_test = {}

for key, values in residual_test.items():
    # Retrieve the mean and standard deviation for the current key
    mean = means_dict[key]
    std = stds_dict[key]

    # Adjust the values by subtracting the mean and dividing by the standard deviation
    adjusted_values = (values - mean) / std

    # Store the adjusted values
    adjusted_residual_test[key] = adjusted_values

##Make predictions using LSTM on stationary/residual part

In [ ]:
import pandas as pd
import numpy as np

def prepare_lstm_input(residuals, n_steps):
    X = []
    for i in range(len(residuals) - n_steps + 1):
        X.append(residuals[i:i + n_steps])
    return np.array(X)

def reverse_scale_to_original(X_scaled, X_min, X_max):
    return ((X_scaled + 1) / 2) * (X_max - X_min) + X_min


# Create a DataFrame for storing final predictions
final_predictions_df = pd.DataFrame(columns=['UserID', 'Final_Prediction'])

# Iterate over each user and their residuals
for user_id, residuals in scaled_residuals_test.items():
    if user_id in lstm_models:
        X_min = mins_dict[user_id]
        X_max = maxs_dict[user_id]
        # Prepare LSTM input from residuals
        if len(residuals) >= n_steps:
            lstm_input = prepare_lstm_input(residuals, n_steps)
            lstm_input = lstm_input.reshape((-1, n_steps, 1))  # Reshape for LSTM

            # Make prediction with LSTM
            lstm_pred_normalized = lstm_models[user_id].predict(lstm_input)

            # Reverse normalization on LSTM predictions to bring them back to original scale
            lstm_pred = reverse_scale_to_original(lstm_pred_normalized, X_min, X_max)

            # Retrieve the corresponding non-stationary model predictions
            non_stationary_pred = user_predictions_non_stationary[user_id]

            # Add LSTM predictions to non-stationary model predictions
            # Assuming non_stationary_pred is aligned with the last lstm_pred
            #combined_pred = non_stationary_pred[-len(lstm_pred):] + (beta_hat[-len(lstm_pred):].T@lstm_pred).flatten()
            combined_pred = non_stationary_pred[-len(lstm_pred):] + lstm_pred.flatten()

        else:
            combined_pred = residuals
    else:
        # If no LSTM model, use non-stationary model predictions as is
        combined_pred = user_predictions_non_stationary[user_id]

    # Add combined predictions to the DataFrame
    final_predictions_df = final_predictions_df.append(pd.DataFrame({
        'UserID': user_id,
        'Final_Prediction': combined_pred
    }), ignore_index=True)

# final_predictions_df now contains combined predictions for each user


In [ ]:
import pandas as pd
import numpy as np

def prepare_lstm_input(residuals, n_steps):
    X = []
    for i in range(len(residuals) - n_steps + 1):
        X.append(residuals[i:i + n_steps])
    return np.array(X)

def reverse_normalize_data(normalized_data, mean, std):
    return (normalized_data * std) + mean

# Create a DataFrame for storing final predictions
final_predictions_df = pd.DataFrame(columns=['UserID', 'Final_Prediction'])

# Iterate over each user and their residuals
for user_id, residuals in adjusted_residual_test.items():
    if user_id in lstm_models:
        mean = means_dict[user_id]
        std = stds_dict[user_id]
        # Prepare LSTM input from residuals
        if len(residuals) >= n_steps:
            lstm_input = prepare_lstm_input(residuals, n_steps)
            lstm_input = lstm_input.reshape((-1, n_steps, 1))  # Reshape for LSTM

            # Make prediction with LSTM
            lstm_pred_normalized = lstm_models[user_id].predict(lstm_input)

            # Reverse normalization on LSTM predictions to bring them back to original scale
            lstm_pred = reverse_normalize_data(lstm_pred_normalized, mean, std)

            # Retrieve the corresponding non-stationary model predictions
            non_stationary_pred = user_predictions_non_stationary[user_id]

            # Add LSTM predictions to non-stationary model predictions
            # Assuming non_stationary_pred is aligned with the last lstm_pred
            #combined_pred = non_stationary_pred[-len(lstm_pred):] + (beta_hat[-len(lstm_pred):].T@lstm_pred).flatten()
            combined_pred = non_stationary_pred[-len(lstm_pred):] + lstm_pred.flatten()

        else:
            combined_pred = residuals
    else:
        # If no LSTM model, use non-stationary model predictions as is
        combined_pred = user_predictions_non_stationary[user_id]

    # Add combined predictions to the DataFrame
    final_predictions_df = final_predictions_df.append(pd.DataFrame({
        'UserID': user_id,
        'Final_Prediction': combined_pred
    }), ignore_index=True)

# final_predictions_df now contains combined predictions for each user


#SMAPE of hybrid model (Rank Decomposition+LSTM hybrid)

In [ ]:
df = pd.DataFrame(df_valid)

# Convert DataFrame to dictionary with UserID as key and Time values as list
true_values = df.groupby('UserID')['Load'].apply(list).to_dict()

In [ ]:
import numpy as np
import pandas as pd

def calculate_smape(actual, predicted):
    """Calculate SMAPE between two series."""
    denominator = (np.abs(actual) + np.abs(predicted))
    diff = np.abs(actual - predicted) / denominator
    diff[denominator == 0] = 0.0  # handle division by zero
    return 100 * np.mean(diff)

# Dictionary to store SMAPE for each user
smape_values_non_stationary = {}

# Iterate over each user
for user in user_predictions_non_stationary:
    # Retrieve the predicted values for the user and convert to a Pandas Series if not already
    predicted = pd.Series(user_predictions_non_stationary[user])

    # Retrieve the true values for the user and convert to a Pandas Series
    actual = pd.Series(true_values[user],index=predicted.index)

    # Calculate SMAPE
    smape = calculate_smape(actual, predicted)

    # Store the SMAPE value
    smape_values_non_stationary[user] = smape

# smape_values dictionary now contains the SMAPE for each user


In [ ]:
final_predictions_df = final_predictions_df.reset_index(drop=True)
sorted_JPL_test = df_valid.reset_index(drop=True)
combined_df = pd.concat([final_predictions_df, sorted_JPL_test], axis=1)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

In [ ]:
import pandas as pd
df = pd.DataFrame(combined_df)

# Function to calculate SMAPE
def calculate_smape(df):
    def smape(y_true, y_pred):
        denominator = (abs(y_true) + abs(y_pred))
        diff = abs(y_true - y_pred) / denominator
        return 100 * diff.mean()

    smape_values = df.groupby('UserID').apply(lambda x: smape(x['Load'], x['Final_Prediction']))
    return smape_values

smape_results_combined = calculate_smape(df)


In [ ]:
# Assuming smape_results1 and smape_results2 are the SMAPE results from two different DataFrames
final_smape_results = pd.DataFrame({
    'SMAPE1': smape_values_non_stationary,
    'SMAPE2': smape_results_combined
})

# Select the better SMAPE for each UserID
final_smape_results['Best_SMAPE'] = final_smape_results.min(axis=1)

# Display the final SMAPE results
print(final_smape_results)

In [ ]:
mean_best_smape = final_smape_results['Best_SMAPE'].mean()
print("SMAPE for hybrid SAMoSSA and LSTM model (%):", mean_best_smape)

In [ ]:
mean_best_smape = final_smape_results['SMAPE2'].mean()
print("SMAPE for hybrid SAMoSSA and LSTM model (%):", mean_best_smape)